# Performance visualization
Draw one combined latency chart (Avg and P50) and one memory chart.
Colors compare Control vs Experiment; columns faceted by bit-width.
- Latency (Avg & P50)
- Peak Memory (MB)

In [ ]:
import pandas as pd
import altair as alt

_df = pd.read_csv("results.csv")

df_long_latency = pd.melt(
    _df,
    id_vars=["bits", "batch_size", "num_heads", "seq_len", "head_dim", "fp16_ratio", "causal", "iters", "warmup"],
    value_vars=["ctrl_avg_ms", "exp_avg_ms", "ctrl_p50_ms", "exp_p50_ms"],
    var_name="metric",
    value_name="ms",
)

def parse_group(metric: str) -> str:
    return "Control" if metric.startswith("ctrl_") else "Experiment"

def parse_stat(metric: str) -> str:
    if "avg" in metric:
        return "avg"
    if "p50" in metric:
        return "p50"
    return metric

df_long_latency["group"] = df_long_latency["metric"].map(parse_group)
df_long_latency["stat"] = df_long_latency["metric"].map(parse_stat)

def plot_latency_by_stat(df_latency: pd.DataFrame, stat: str):
    title_map = {"avg": "Avg", "p50": "P50"}
    df = df_latency[df_latency["stat"] == stat]
    chart = alt.Chart(df).mark_bar().encode(
        y=alt.Y("group:N", title="Group", sort=["Control", "Experiment"]),
        x=alt.X("ms:Q", title="Latency (ms)"),
        color=alt.Color("group:N", title="Group"),
        column=alt.Column("bits:N", title="Bit-width (bits)"),
    ).properties(title=f"Latency ({title_map.get(stat, stat)})")
    return chart

df_long_mem = pd.melt(
    _df,
    id_vars=["bits", "batch_size", "num_heads", "seq_len", "head_dim", "fp16_ratio", "causal", "iters", "warmup"],
    value_vars=["ctrl_peak_mem", "exp_peak_mem"],
    var_name="metric",
    value_name="bytes",
)

df_long_mem["group"] = df_long_mem["metric"].map(parse_group)
df_long_mem["MB"] = df_long_mem["bytes"] / (1024 * 1024)

def plot_peak_memory(df_memory: pd.DataFrame):
    chart = alt.Chart(df_memory).mark_bar().encode(
        y=alt.Y("group:N", title="Group", sort=["Control", "Experiment"]),
        x=alt.X("MB:Q", title="Peak Memory (MB)"),
        color=alt.Color("group:N", title="Group"),
        column=alt.Column("bits:N", title="Bit-width (bits)"),
    ).properties(title="Peak Memory")
    return chart

(plot_latency_by_stat(df_long_latency, "avg") & plot_latency_by_stat(df_long_latency, "p50")) & plot_peak_memory(df_long_mem)


alt.VConcatChart(...)